In [1]:
!pip install -q openai
!pip install -q langchain langchain-community
!pip install -q langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 978.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.9/389.9 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 24.3 MB/s eta 0:00:00


In [2]:
import openai
from langchain_openai import ChatOpenAI
from langchain import LLMMathChain
from langchain.agents import initialize_agent, Tool
import json
from google.colab import userdata


In [ ]:
# Initialize ChatOpenAI (GPT-4o-mini) using the langchain_openai package
llm = ChatOpenAI(
    model_name="gpt-4o-mini",
    openai_api_key=userdata.get('open_ai'),
    temperature=0.7
)


In [ ]:
from langchain_core.tools import tool


@tool
def add(a: int, b: int) -> int:
    """Adds a and b."""
    return a + b


@tool
def multiply(a: int, b: int) -> int:
    """Multiplies a and b."""
    return a * b


tools = [add, multiply]

llm_with_tools = llm.bind_tools(tools)

In [ ]:
from langchain_core.messages import HumanMessage

query = "What is 3 * 12? Also, what is 11 + 49?"

messages = [HumanMessage(query)]

#where we get what all tools need to be invoked
ai_msg = llm_with_tools.invoke(messages)

print(ai_msg.tool_calls)

messages.append(ai_msg)

[{'name': 'multiply', 'args': {'a': 3, 'b': 12}, 'id': 'call_f4psCSeq3tCJJEFK8bQsNjVi', 'type': 'tool_call'}, {'name': 'add', 'args': {'a': 11, 'b': 49}, 'id': 'call_5T5hWmHiY2pd8BKw9yZxCX0R', 'type': 'tool_call'}]


In [ ]:
for tool_call in ai_msg.tool_calls:
    selected_tool = {"add": add, "multiply": multiply}[tool_call["name"].lower()]
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)

messages

[HumanMessage(content='What is 3 * 12? Also, what is 11 + 49?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_f4psCSeq3tCJJEFK8bQsNjVi', 'function': {'arguments': '{"a": 3, "b": 12}', 'name': 'multiply'}, 'type': 'function'}, {'id': 'call_5T5hWmHiY2pd8BKw9yZxCX0R', 'function': {'arguments': '{"a": 11, "b": 49}', 'name': 'add'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 50, 'prompt_tokens': 87, 'total_tokens': 137, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0705bf87c0', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-9acec9c3-8e5a-4b5d-b3d6-c210728b08dc-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 12}, 'id': 'call_f4p

In [ ]:
response = llm_with_tools.invoke(messages)

In [ ]:
print(response.content)

The result of \(3 \times 12\) is 36, and the result of \(11 + 49\) is 60.


In [ ]:
from openai import OpenAI

client = OpenAI(api_key=userdata.get('open_ai'))

tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {"type": "string"},
                    "unit": {"type": "string", "enum": ["c", "f"]},
                },
                "required": ["location", "unit"],
                "additionalProperties": False,
            },
        },
    }
]

completion = client.chat.completions.create(
    model="gpt-4o",
    messages=[{"role": "user", "content": "What's the weather like in Paris today?"}],
    tools=tools,
)

print(completion.choices[0].message.tool_calls)

[ChatCompletionMessageToolCall(id='call_fR7hNkd9o3JBUtgeFmOXzCgI', function=Function(arguments='{"location":"Paris","unit":"c"}', name='get_weather'), type='function')]


In [ ]:
from openai import OpenAI
import json

client = OpenAI(api_key=userdata.get('open_ai'))

# Define functions
tools = [
    {
        "type": "function",
        "function": {
            "name": "generate_recipe",
            "description": "Generate a recipe based on the user's input",
            "parameters": {
                "type": "object",
                "properties": {
                    "title": {
                        "type": "string",
                        "description": "Title of the recipe.",
                    },
                    "ingredients": {
                        "type": "array",
                        "items": {"type": "string"},
                        "description": "List of ingredients required for the recipe.",
                    },
                    "instructions": {
                        "type": "array",
                        "items": {"type": "string"},
                        "description": "Step-by-step instructions for the recipe.",
                    },
                },
                "required": ["title", "ingredients", "instructions"],
                "additionalProperties": False,
            },
        },
    }
]

response_stream = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {
            "role": "system",
            "content": (
                "You are an expert cook who can help turn any user input into a delicious recipe."
                "As soon as the user tells you what they want, use the generate_recipe tool to create a detailed recipe for them."
            ),
        },
        {
            "role": "user",
            "content": "I want to make pancakes for 4.",
        },
    ],
    tools=tools,
    stream=True,
)

function_arguments = ""
function_name = ""
is_collecting_function_args = False

for part in response_stream:
    delta = part.choices[0].delta
    finish_reason = part.choices[0].finish_reason

    # Process assistant content
    if 'content' in delta:
        print("Assistant:", delta.content)

    if delta.tool_calls:
        is_collecting_function_args = True
        tool_call = delta.tool_calls[0]

        if tool_call.function.name:
            function_name = tool_call.function.name
            print(f"Function name: '{function_name}'")

        # Process function arguments delta
        if tool_call.function.arguments:
            function_arguments += tool_call.function.arguments
            print(f"Arguments: {function_arguments}")

    # Process tool call with complete arguments
    if finish_reason == "tool_calls" and is_collecting_function_args:
        print(f"Function call '{function_name}' is complete.")
        args = json.loads(function_arguments)
        print("Complete function arguments:")
        print(json.dumps(args, indent=2))

        # Reset for the next potential function call
        function_arguments = ""
        function_name = ""
        is_collecting_function_args = False

Function name: 'generate_recipe'
Arguments: {"
Arguments: {"title
Arguments: {"title":"
Arguments: {"title":"P
Arguments: {"title":"Panc
Arguments: {"title":"Pancakes
Arguments: {"title":"Pancakes for
Arguments: {"title":"Pancakes for 
Arguments: {"title":"Pancakes for 4
Arguments: {"title":"Pancakes for 4","
Arguments: {"title":"Pancakes for 4","ingredients
Arguments: {"title":"Pancakes for 4","ingredients":["
Arguments: {"title":"Pancakes for 4","ingredients":["2
Arguments: {"title":"Pancakes for 4","ingredients":["2 cups
Arguments: {"title":"Pancakes for 4","ingredients":["2 cups all
Arguments: {"title":"Pancakes for 4","ingredients":["2 cups all-purpose
Arguments: {"title":"Pancakes for 4","ingredients":["2 cups all-purpose flour
Arguments: {"title":"Pancakes for 4","ingredients":["2 cups all-purpose flour","
Arguments: {"title":"Pancakes for 4","ingredients":["2 cups all-purpose flour","2
Arguments: {"title":"Pancakes for 4","ingredients":["2 cups all-purpose flour","2 tablespoons